In [115]:
import time

import tensorflow as tf
import numpy as np


In [116]:
batch_size = 10
input_shape = [batch_size, 256, 256, 3]
ph = tf.placeholder(tf.float32, input_shape)

In [130]:
def build(img, debugSess=None):
    assert input_shape == [d.value for d in img.get_shape()]   # now we fix the image size and assume 
      # img has an already-defined shape. Otherwise Tensorflow will fail to infer the output shape of the layer
        # which is very inconvinent for debugging
    
    # img = tf.pad(img, [[40,40],[40,40]]) # TODO maybe add it later after I finished it. 
            # But I don't know why fast-style-transfer repo doesn't follow the paper's supp material PDF to
            #    (1) have this padding step
            #    (2) not use padding in the residual blocks' conv layers
            # Maybe I can try to implement it later to see if it can produce better images
    conv1 = conv_layer(img, n_in_channel=3, n_out_channel=32, filter_size=9, stride=1, hasRelu=True)
    conv2 = conv_layer(conv1, n_in_channel=32, n_out_channel=64, filter_size=3, stride=2, hasRelu=True)
    conv3 = conv_layer(conv2, n_in_channel=64, n_out_channel=128, filter_size=3, stride=2, hasRelu=True)
    res1 = residual_block(conv3, n_in_channel=128, n_out_channel=128)
    res2 = residual_block(res1, n_in_channel=128, n_out_channel=128)
    res3 = residual_block(res2, n_in_channel=128, n_out_channel=128)
    res4 = residual_block(res3, n_in_channel=128, n_out_channel=128)
    res5 = residual_block(res4, n_in_channel=128, n_out_channel=128)
    deconv1 = de_conv_layer(res5, n_in_channel=128, n_out_channel=64, filter_size=3, stride=2)
    deconv2 = de_conv_layer(deconv1, n_in_channel=64, n_out_channel=32, filter_size=3, stride=2)
    convColor = conv_layer(deconv2, n_in_channel=32, n_out_channel=3, filter_size=9, stride=1)
    tanh = tf.nn.tanh(convColor) * 150 + 255./2  # TODO: why tanh * 150 ? why + 255/2 ? 
    
    return tanh
    
def conv_layer(input, n_in_channel, n_out_channel, filter_size, stride, hasRelu=True):
    # TODO conv layer without adding bias ( bias is not used in paper either). but I could try it later if time permitted. tf.nn.bias_add
    filt = tf.Variable(tf.truncated_normal([filter_size, filter_size, n_in_channel, n_out_channel], stddev=.1))
    output = tf.nn.conv2d(input, filt, [1,stride,stride,1], padding='SAME')
    output = _instance_norm(output) # TODO read what is instance normalization 
    if hasRelu:
        output = tf.nn.relu(output)
    print("conv layer, output size: %s" % ([i.value for i in output.get_shape()]))
    return output

def de_conv_layer(input, n_in_channel, n_out_channel, filter_size, stride):
    filt = tf.Variable(tf.truncated_normal([filter_size, filter_size, n_out_channel, n_in_channel], stddev=.1))
    in_shape = [s.value for s in input.get_shape()]
    out_shape = [in_shape[0], in_shape[1]*stride, in_shape[2]*stride, n_out_channel]
    output = tf.nn.conv2d_transpose(input, filt, output_shape=out_shape, strides=[1,stride, stride, 1])
    print("deconv layer, output size: %s" % ([i.value for i in output.get_shape()]))
    return output

def residual_block(input, n_in_channel, n_out_channel, name="n/a"):
    print("START residual_block ")
    output = conv_layer(input, n_in_channel, n_out_channel, filter_size=3, stride=1, hasRelu=True)
    output = conv_layer(output,n_out_channel, n_out_channel, filter_size=3, stride=1, hasRelu=False)
    output = input + output
    print("END residual_block")
    return output
    

# TODO Read what is instance normalization. The following code is copied, I don't know how it works
def _instance_norm(net):
    batch, rows, cols, channels = [i.value for i in net.get_shape()]
    var_shape = [channels]
    mu, sigma_sq = tf.nn.moments(net, [1,2], keep_dims=True)
    shift = tf.Variable(tf.zeros(var_shape))
    scale = tf.Variable(tf.ones(var_shape))
    epsilon = 1e-3
    normalized = (net-mu)/(sigma_sq + epsilon)**(.5)
    return scale * normalized + shift


In [131]:
input = tf.placeholder(tf.float32, input_shape)

build(input)

conv layer, output size: [10, 256, 256, 32]
conv layer, output size: [10, 128, 128, 64]
conv layer, output size: [10, 64, 64, 128]
START residual_block 
conv layer, output size: [10, 64, 64, 128]
conv layer, output size: [10, 64, 64, 128]
END residual_block
START residual_block 
conv layer, output size: [10, 64, 64, 128]
conv layer, output size: [10, 64, 64, 128]
END residual_block
START residual_block 
conv layer, output size: [10, 64, 64, 128]
conv layer, output size: [10, 64, 64, 128]
END residual_block
START residual_block 
conv layer, output size: [10, 64, 64, 128]
conv layer, output size: [10, 64, 64, 128]
END residual_block
START residual_block 
conv layer, output size: [10, 64, 64, 128]
conv layer, output size: [10, 64, 64, 128]
END residual_block
deconv layer, output size: [10, 128, 128, 64]
deconv layer, output size: [10, 256, 256, 32]
conv layer, output size: [10, 256, 256, 3]


<tf.Tensor 'add_653:0' shape=(10, 256, 256, 3) dtype=float32>

(1, 4, 4, 1)
[[[[ 1.]
   [ 1.]
   [ 2.]
   [ 1.]]

  [[ 1.]
   [ 1.]
   [ 2.]
   [ 1.]]

  [[ 2.]
   [ 2.]
   [ 4.]
   [ 2.]]

  [[ 1.]
   [ 1.]
   [ 2.]
   [ 1.]]]]
